In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [5]:
df = pd.read_excel(r"/content/drive/MyDrive/Colab Notebooks/深度學習/上證指數.xlsx") #存成dataframe的物件,r:regular
df

,名稱,代碼,日期,開盤,最高,最低,收盤,漲跌
0,上證指數,980001,2017-01-03,3105.31,3136.46,3105.31,3135.92,1
1,上證指數,980001,2017-01-04,3133.79,3160.10,3130.11,3158.79,1
2,上證指數,980001,2017-01-05,3157.91,3168.50,3154.28,3165.41,1
3,上證指數,980001,2017-01-06,3163.78,3172.03,3153.03,3154.32,0
4,上證指數,980001,2017-01-09,3148.53,3173.14,3147.74,3171.24,1
...,...,...,...,...,...,...,...,...
177,上證指數,980001,2017-09-21,3364.70,3377.88,3356.88,3357.81,0
178,上證指數,980001,2017-09-22,3347.16,3356.45,3334.98,3352.53,0
179,上證指數,980001,2017-09-25,3344.59,3350.96,3334.94,3341.55,0
180,上證指數,980001,2017-09-26,3336.35,3347.16,3332.60,3343.58,1


In [7]:
import torch
import torch.utils.data as Data
import numpy as np  #先將datafram轉numpy再轉tensor

In [28]:
class MyDataset(Data.Dataset):
  def __init__(self, file_path, seq_length, train=True, transform=None): #seq_length(每固定一段長度為一組)
    pd.read_excel(file_path) #存成dataframe的物件,r:regular
    df_len = len(df)-1
    df_len -= (df_len%seq_length) #把剩下不夠成為一組的資料刪除
    total_size = df_len//seq_length
    train_size = int(total_size*0.7) #捨去小數點

    f = df.iloc[:(train_size*seq_length),3:7] #留3~6column
    l = df.iloc[1:(train_size*seq_length)+1,7:] 
    data_size = train_size
    if train == False:
      f = df.iloc[(train_size*seq_length):df_len,3:7]
      l = df.iloc[(train_size*seq_length)+1:df_len+1,7:] 
      data_size = total_size - train_size
    self.features = np.array(f).astype("float32").reshape(data_size,seq_length,4) #要強制轉型成float32晚點才能轉tensor
    self.labels = np.array(l).astype("int").reshape(data_size,seq_length,1)
    self.transform = transform

  def __len__(self):  #資料長度
    return len(self.labels)
  
  def __getitem__(self, idx): #取資料，idx(第幾筆)
    if torch.is_tensor(idx):
      idx = idx.tolist()
    X = self.features[idx]
    Y = self.labels[idx]

    if self.transform:
      X = self.transform(X)
      Y = self.transform(Y) 
    return X,Y

In [32]:
def toTensor(x):
  return torch.tensor(x)

In [33]:
train_dataset = MyDataset(file_path = r"/content/drive/MyDrive/Colab Notebooks/深度學習/上證指數.xlsx",
              seq_length = 5,
              train = True,
              transform = toTensor)

test_dataset = MyDataset(file_path = r"/content/drive/MyDrive/Colab Notebooks/深度學習/上證指數.xlsx",
              seq_length = 5,
              train = False,
              transform = toTensor)

In [36]:
features, labels = train_dataset[2:9]
print(features.shape)
print(labels.shape)
features

torch.Size([7, 5, 4])
torch.Size([7, 5, 1])


tensor([[[3087.0300, 3108.9099, 3072.3401, 3108.7700],
         [3104.7700, 3123.7200, 3098.5901, 3113.0100],
         [3104.9700, 3115.7800, 3094.0100, 3101.3000],
         [3095.8201, 3125.6599, 3095.2100, 3123.1399],
         [3125.4199, 3145.8401, 3125.4199, 3136.7700]],

        [[3134.5901, 3149.5300, 3131.2200, 3142.5500],
         [3137.6499, 3151.4700, 3133.1899, 3149.5500],
         [3149.2200, 3163.1001, 3148.9099, 3159.1699],
         [3160.0801, 3162.6799, 3136.0100, 3140.1699],
         [3143.0901, 3158.8401, 3135.3899, 3156.9800]],

        [[3154.3999, 3159.5400, 3140.0400, 3153.0901],
         [3148.0901, 3167.4500, 3132.0300, 3166.9800],
         [3164.6899, 3186.8401, 3162.5701, 3183.1799],
         [3183.0100, 3205.0500, 3182.8000, 3196.7000],
         [3198.9900, 3219.4099, 3198.9900, 3216.8401]],

        [[3216.1399, 3219.3999, 3205.2900, 3217.9299],
         [3215.4600, 3236.0000, 3206.5601, 3212.9900],
         [3210.3601, 3230.2800, 3207.7900, 3229.6201],
    

In [39]:
train_loader = Data.DataLoader(dataset= train_dataset,
                batch_size = 5,
                shuffle=True)
test_loader = Data.DataLoader(dataset= train_dataset,
                batch_size = 5,
                shuffle=False)

In [56]:
for i,(seqs, labels) in enumerate(train_loader): #檢查loader
  print(seqs.shape)
seqs

torch.Size([5, 5, 4])
torch.Size([5, 5, 4])
torch.Size([5, 5, 4])
torch.Size([5, 5, 4])
torch.Size([5, 5, 4])


tensor([[[3105.3101, 3136.4600, 3105.3101, 3135.9199],
         [3133.7900, 3160.1001, 3130.1101, 3158.7900],
         [3157.9099, 3168.5000, 3154.2800, 3165.4099],
         [3163.7800, 3172.0300, 3153.0300, 3154.3201],
         [3148.5300, 3173.1399, 3147.7400, 3171.2400]],

        [[3148.0200, 3150.4600, 3134.6101, 3140.0100],
         [3148.9900, 3157.0300, 3132.6201, 3156.2100],
         [3152.2400, 3186.9800, 3146.6399, 3147.4500],
         [3138.4399, 3158.0500, 3118.0901, 3157.8701],
         [3157.0000, 3187.8899, 3156.9800, 3185.4399]],

        [[3183.4199, 3193.4600, 3172.4600, 3191.2000],
         [3183.6299, 3193.4399, 3170.7900, 3173.2000],
         [3174.9800, 3188.7700, 3174.2800, 3188.0601],
         [3176.9500, 3193.2400, 3171.5701, 3192.4299],
         [3192.0000, 3196.2900, 3177.0200, 3195.9099]],

        [[3265.6299, 3265.6299, 3246.0901, 3252.9500],
         [3252.8701, 3262.1001, 3233.2800, 3241.3101],
         [3235.1399, 3240.0200, 3195.8501, 3210.2400],
    

# RNN

In [68]:
input_size = 4
hidden_size = 32 #memory cell
num_layers = 1
num_class = 1

In [69]:
import torch.nn as nn
from torch.autograd import Variable

In [73]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_class):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.normal = nn.BatchNorm1d(5,affine=True) #正規化(5:batch_size)
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, num_class)
    self.sigmoid = nn.Sigmoid()
  def forward(self, x):
    h0 = Variable(torch.zeros(num_layers, x.size(0), hidden_size))
    c0 = Variable(torch.zeros(num_layers, x.size(0), hidden_size))
    out = self.normal(x)
    out, _ = self.lstm(out, (h0,c0))
    out = self.fc(out)
    out = self.sigmoid(out)
    return out

In [74]:
rnn = RNN(input_size, hidden_size, num_layers, num_class)

rnn

RNN(
  (normal): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(4, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [75]:
output = rnn(seqs)
output

tensor([[[0.5002],
         [0.4943],
         [0.4918],
         [0.4884],
         [0.4879]],

        [[0.5082],
         [0.5053],
         [0.5030],
         [0.5000],
         [0.4974]],

        [[0.5005],
         [0.4990],
         [0.5019],
         [0.5012],
         [0.5052]],

        [[0.5034],
         [0.4987],
         [0.4973],
         [0.4988],
         [0.4964]],

        [[0.5112],
         [0.5085],
         [0.5070],
         [0.5061],
         [0.5050]]], grad_fn=<SigmoidBackward0>)

In [76]:
import torch.optim as opt

In [77]:
loss_fn = nn.BCELoss()
optimizer = opt.Adam(rnn.parameters(), lr=0.001)

In [80]:
for epoch in range(800):
  for i, (seqs, labels) in enumerate(train_loader):
    seqs = Variable(seqs)
    labels = Variable(labels)
    optimizer.zero_grad()
    outputs = rnn(seqs)
    loss = loss_fn(outputs,labels.float())
    loss.backward()
    optimizer.step()

    if (epoch+1)%100 == 0:
      print("Epoch: %d, batch: %d, Loss: %.4f" %(epoch+1, i+1, loss.data))


Epoch: 100, batch: 1, Loss: 0.6271
Epoch: 100, batch: 2, Loss: 0.6542
Epoch: 100, batch: 3, Loss: 0.6240
Epoch: 100, batch: 4, Loss: 0.6371
Epoch: 100, batch: 5, Loss: 0.5839
Epoch: 200, batch: 1, Loss: 0.6094
Epoch: 200, batch: 2, Loss: 0.5431
Epoch: 200, batch: 3, Loss: 0.6678
Epoch: 200, batch: 4, Loss: 0.6169
Epoch: 200, batch: 5, Loss: 0.6516
Epoch: 300, batch: 1, Loss: 0.6177
Epoch: 300, batch: 2, Loss: 0.5623
Epoch: 300, batch: 3, Loss: 0.5961
Epoch: 300, batch: 4, Loss: 0.6912
Epoch: 300, batch: 5, Loss: 0.6131
Epoch: 400, batch: 1, Loss: 0.5209
Epoch: 400, batch: 2, Loss: 0.6110
Epoch: 400, batch: 3, Loss: 0.6924
Epoch: 400, batch: 4, Loss: 0.6014
Epoch: 400, batch: 5, Loss: 0.5882
Epoch: 500, batch: 1, Loss: 0.5674
Epoch: 500, batch: 2, Loss: 0.6521
Epoch: 500, batch: 3, Loss: 0.6834
Epoch: 500, batch: 4, Loss: 0.5815
Epoch: 500, batch: 5, Loss: 0.6503
Epoch: 600, batch: 1, Loss: 0.5834
Epoch: 600, batch: 2, Loss: 0.4733
Epoch: 600, batch: 3, Loss: 0.5639
Epoch: 600, batch: 4

In [82]:
correct = 0 
total = 0
for seqs, labels in test_loader:
  seqs = Variable(seqs)
  outputs = rnn(seqs)
  predicted = outputs.gt(0.5) #>0.5就是True
  total += (labels.size(0)*labels.size(1))
  correct += (predicted.int() == labels).sum()
print("Accuracy: %.3f%%" %(100*float(correct)//float(total)))

Accuracy: 59.000%
